In [1]:
import json
import pandas as pd

In [2]:
mini_json_filename = 'results_20210718/ph5_19-003_example/miniPH5_00001_ref_fs.json'

In [3]:
def not_data_key(key : str) -> bool:
    """
    This function looks for if a key is a known .zXXX and if so returns true
    :param key: dict key from reference json
    :return: Boolean true if is a known zarr key
    """
    is_zattrs = key.endswith('.zattrs')
    is_zarray = key.endswith('.zarray')
    is_zgroup = key.endswith('.zgroup')
    return is_zattrs or is_zarray or is_zgroup

In [4]:
fp = open(mini_json_filename)
mini_json = json.load(fp)
fp.close()

In [5]:
def walk_dict(d: dict, key_filt):
    rtn_d = {}
    for k, v in d.items():
        if key_filt(k):
            continue # filter value by not returning
        if isinstance(v, dict):
            sub_dict = walk_dict(v, key_filt)
            if len(sub_dict) > 0: # only assign if this subdict was not empty
                rtn_d.setdefault(k, sub_dict)
        else:
            rtn_d.setdefault(k, v)

    return rtn_d

data_dict = walk_dict(mini_json, not_data_key)

In [6]:
df = pd.DataFrame(data_dict['refs'])
df = df.transpose()
df = df.reset_index()
df.columns = ['path', 'prefix', 'offset', 'size']

In [7]:
split_key = df['path'].str.split('/')
# IMPROVE handling if data is not at the same depth of path (n*'/')
key_df = pd.DataFrame(split_key.to_list()) # convert to 2D df

df['das_name'] = key_df[2].to_numpy()
df['data_name'] = key_df[3].to_numpy()
df['chunk_num'] = key_df[4].to_numpy()
df

,path,prefix,offset,size,das_name,data_name,chunk_num
0,Experiment_g/Maps_g/Das_g_1X101/Hdr_a_0001/0,{{u}},1986632,2694,Das_g_1X101,Hdr_a_0001,0
1,Experiment_g/Maps_g/Das_g_1X101/Hdr_a_0002/0,{{u}},9400674,2911,Das_g_1X101,Hdr_a_0002,0
2,Experiment_g/Receivers_g/Das_g_1X101/Data_a_00...,{{u}},103968,34386,Das_g_1X101,Data_a_0001,0
3,Experiment_g/Receivers_g/Das_g_1X101/Data_a_00...,{{u}},138354,34273,Das_g_1X101,Data_a_0001,1
4,Experiment_g/Receivers_g/Das_g_1X101/Data_a_00...,{{u}},446154,34082,Das_g_1X101,Data_a_0001,10
...,...,...,...,...,...,...,...
327,Experiment_g/Receivers_g/Das_g_1X101/Data_a_00...,{{u}},9400579,95,Das_g_1X101,Data_a_0006,54
328,Experiment_g/Receivers_g/Das_g_1X101/Data_a_00...,{{u}},8385343,33087,Das_g_1X101,Data_a_0006,6
329,Experiment_g/Receivers_g/Das_g_1X101/Data_a_00...,{{u}},8418430,32742,Das_g_1X101,Data_a_0006,7
330,Experiment_g/Receivers_g/Das_g_1X101/Data_a_00...,{{u}},8451172,33001,Das_g_1X101,Data_a_0006,8


In [8]:
df.groupby('data_name')['size'].sum()

data_name
Data_a_0001    1879640
Data_a_0002    1244348
Data_a_0003    1876519
Data_a_0004    1243070
Data_a_0005    1820589
Data_a_0006    1211238
Hdr_a_0001        2694
Hdr_a_0002        2911
Name: size, dtype: object

In [9]:
df['size'].sum()


9281009